In [47]:
library(tidytree)
library(tidyverse)
library(ape)
library(castor)
options(repr.plot.width = 8, repr.plot.height = 16, repr.plot.res = 100)

── Attaching core tidyverse packages ──────────────────────────────────────────────────────────────── tidyverse 2.0.0 ──
✔ forcats   1.0.0     ✔ readr     2.1.4
✔ ggplot2   3.4.4     ✔ stringr   1.5.0
✔ lubridate 1.9.3     ✔ tibble    3.2.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks tidytree::filter(), stats::filter()
✖ dplyr::lag()    masks stats::lag()
✖ ape::where()    masks dplyr::where()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [48]:
ref_tree <- "../Result_rona/Diff_tree_size_p__Campylobacterota/p__Campylobacterota_300/ref_tree.tre"
loop1_tree <- "../Result_rona/Diff_tree_size_p__Campylobacterota/p__Campylobacterota_300/loop_1/tree_update/p__Campylobacterota_300_1.treefile"
loop2_tree <- "../Result_rona/Diff_tree_size_p__Campylobacterota/p__Campylobacterota_300/loop_2/tree_update/p__Campylobacterota_300_2.treefile"

ref_tree <- read.tree(ref_tree)
loop1_tree <- read.tree(loop1_tree)
loop2_tree <- read.tree(loop2_tree)

In [49]:
contree <- consensus(ref_tree, loop1_tree)

In [50]:
gtdb_tree <- "../data/r207/gtdb_r207_bac120_unscaled.decorated.tree"
gtdb_tree <- ape::read.tree(gtdb_tree)
gtdb_treedf <- as_tibble(gtdb_tree)

In [73]:
taxon_node <- gtdb_treedf %>% 
    filter(!is.na(branch.length)) %>% 
    filter(node > length(gtdb_tree$tip.label))  %>% 
    filter(nzchar(label) > 0) %>%
    filter(is.na(suppressWarnings(as.numeric(label)))) %>%
    mutate(label = trimws(label, "both", "'")) %>% 
    separate(label, into = c("support_value", "taxa_info"), sep = ":", extra = "merge") %>%
    mutate(taxa_info = strsplit(taxa_info, "; ")) %>%
    unnest(taxa_info)

! # Invaild edge matrix for <phylo>. A <tbl_df> is returned.



In [74]:
taxon_node %>% filter(taxa_info == "p__Campylobacterota")

parent,node,branch.length,support_value,taxa_info
<int>,<int>,<dbl>,<chr>,<chr>
63138,63199,0.06589,86.0,p__Campylobacterota


In [79]:
all_subtrees <- taxon_node %>% 
    mutate(subtree = get_subtrees_at_nodes(tree = gtdb_tree,node = node - length(gtdb_tree$tip.label))$subtree)

In [99]:
 all_subtrees %>% 
    mutate(tips = map_dbl(subtree, ~ .x$tip.label)) %>% 
    select(-subtree)

ERROR: [1m[33mError[39m in `mutate()`:[22m
[1m[22m[36mℹ[39m In argument: `tips = map_dbl(subtree, ~.x$tip.label)`.
[1mCaused by error in `map_dbl()`:[22m
[1m[22m[36mℹ[39m In index: 1.
[1mCaused by error:[22m
[33m![39m Result must be length 1, not 695.
